In [1]:
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import gensim
from gensim import corpora
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nuzzul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
data = pd.read_csv("/Users/nuzzul/Downloads/BT4222 LDA/cleaned_paradox-sg-merchant-court.csv")
data.head()

,Unnamed: 0,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,year,stem_review,lem_review
0,0,Scenic27042346319,My F1 Trip staying place September 2023,The Hotel front-desk was very nice and welcomi...,Trip type: Travelled with friends,NaN,4.0,NaN,5.0,True,Positive,f trip staying place september hotel front des...,My F1 Trip staying place September 2023 The Ho...,2023-09-01,PostCovid,2023,f trip stay place septemb hotel front desk nic...,f trip stay place september hotel front desk n...
1,1,gmac73,Great location & excellent staff,Fantastic hotel in a great location! Excellent...,Trip type: Travelled with family,"Melbourne, Australia",61.0,43.0,NaN,False,NaN,great location excellent staff fantastic hotel...,Great location & excellent staff Fantastic hot...,2023-04-01,PostCovid,2023,great locat excel staff fantast hotel great lo...,great location excellent staff fantastic hotel...
2,2,williefil13,Comfy hotel,"The hotel is well located downtown , close to...",Trip type: Travelled on business,"Singapore, Singapore",1037.0,187.0,4.0,True,Positive,comfy hotel hotel well located downtown close ...,Comfy hotel The hotel is well located downtow...,2023-09-01,PostCovid,2023,comfi hotel hotel well locat downtown close cl...,comfy hotel hotel well locate downtown close c...
3,3,Poosan,Great Stay During F1 September 2023,We stayed here for 5 nights over the F1 Weeken...,Trip type: Travelled as a couple,NaN,750.0,89.0,5.0,True,Positive,great stay f september stayed nights f weekend...,Great Stay During F1 September 2023 We stayed ...,2023-09-01,PostCovid,2023,great stay f septemb stay night f weekend hote...,great stay f september stay night f weekend ho...
4,4,AJS2015,An excellent hotel!,We stopped over at the Paradox on both legs of...,Trip type: Travelled as a couple,"Calne, United Kingdom",147.0,84.0,5.0,True,Positive,excellent hotel stopped paradox legs holiday a...,An excellent hotel! We stopped over at the Par...,2023-03-01,PostCovid,2023,excel hotel stop paradox leg holiday australia...,excellent hotel stop paradox legs holiday aust...


In [4]:
reviews = data["lem_review"]
reviews.head()

0    f trip stay place september hotel front desk n...
1    great location excellent staff fantastic hotel...
2    comfy hotel hotel well locate downtown close c...
3    great stay f september stay night f weekend ho...
4    excellent hotel stop paradox legs holiday aust...
Name: lem_review, dtype: object

In [5]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([comment.split() for comment in reviews])
# Filter only those that occur at least 10 times
finder.apply_freq_filter(10)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

In [6]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = nltk.collocations.TrigramCollocationFinder.from_documents([comment.split() for comment in reviews])
# Filter only those that occur at least 10 times
finder.apply_freq_filter(10)
trigram_scores = finder.score_ngrams(trigram_measures.pmi)

In [7]:
bigram_pmi = pd.DataFrame(bigram_scores)
bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)
bigram_pmi

,bigram,pmi
0,"(jet, lag)",13.796758
1,"(melissa, hwang)",13.448835
2,"(la, carte)",13.436862
3,"(stone, throw)",13.133793
4,"(fort, canning)",13.086264
...,...,...
1821,"(hotel, hotel)",-1.591414
1822,"(hotel, walk)",-1.643343
1823,"(staff, location)",-1.721491
1824,"(room, location)",-1.727599


In [8]:
trigram_pmi = pd.DataFrame(trigram_scores)
trigram_pmi.columns = ['trigram', 'pmi']
trigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)
trigram_pmi

,trigram,pmi
0,"(kim, melissa, hwang)",26.438238
1,"(hop, hop, bus)",20.011110
2,"(marina, bay, sands)",19.615388
3,"(marina, bay, sand)",19.332988
4,"(ellenborough, market, cafe)",19.231572
...,...,...
369,"(great, hotel, location)",2.849108
370,"(stay, hotel, great)",2.837294
371,"(hotel, great, staff)",2.748927
372,"(hotel, great, hotel)",2.214506


In [9]:
stop_word_list = set(stopwords.words('english'))

In [10]:
# Filter for bigrams with only noun-type structures
def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in stop_word_list or bigram[1] in stop_word_list:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True

In [11]:
# Filter for trigrams with only noun-type structures
def trigram_filter(trigram):
    tag = nltk.pos_tag(trigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['JJ','NN']:
        return False
    if trigram[0] in stop_word_list or trigram[-1] in stop_word_list or trigram[1] in stop_word_list:
        return False
    if 'n' in trigram or 't' in trigram:
         return False
    if 'PRON' in trigram:
        return False
    return True 

In [12]:
# Can set pmi threshold to whatever makes sense - eyeball through and select threshold where n-grams stop making sense
# choose top 500 ngrams in this case ranked by PMI that have noun like structures
filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram: bigram_filter(bigram['bigram']) and bigram.pmi > 5, axis = 1)][:500]

filtered_trigram = trigram_pmi[trigram_pmi.apply(lambda trigram: \
                                                 trigram_filter(trigram['trigram'])\
                                                 and trigram.pmi > 5, axis = 1)][:500]


bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 2 or len(x[1]) > 2]
trigrams = [' '.join(x) for x in filtered_trigram.trigram.values if len(x[0]) > 2 or len(x[1]) > 2 and len(x[2]) > 2]

In [13]:
bigrams[:15]

['jet lag',
 'melissa hwang',
 'la carte',
 'stone throw',
 'fort canning',
 'jia yan',
 'kim melissa',
 'hustle bustle',
 'en route',
 'iron board',
 'sun lounger',
 'exceed expectation',
 'blue potato',
 'air conditioning',
 'live music']

In [14]:
trigrams[:15]

['kim melissa hwang',
 'hop hop bus',
 'marina bay sands',
 'marina bay sand',
 'ellenborough market cafe',
 'nothing much trouble',
 'chinese new year',
 'nespresso coffee machine',
 'go extra mile',
 'swiss tel merchant',
 'free international call',
 'water pressure shower',
 'blue potato restaurant',
 'king size bed',
 'centre across road']

In [15]:
# Concatenate n-grams
def replace_ngram(x):
    for gram in trigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x

In [16]:
reviews_w_ngrams = reviews.copy()
final_reviews = reviews_w_ngrams.map(lambda x: replace_ngram(x))
final_reviews.head()

0    f trip stay place september hotel front_desk n...
1    great_location_excellent staff fantastic hotel...
2    comfy hotel hotel_well_locate downtown close_c...
3    great stay f september stay night f weekend ho...
4    excellent hotel stop paradox legs holiday aust...
Name: lem_review, dtype: object

In [17]:
#final_reviews = final_reviews.apply(lambda x: [x])
final_review = final_reviews.tolist()
final_reviews = ""
for x in final_review:
    final_reviews += x

In [18]:
final_reviews = final_reviews.split()

In [19]:
x = []
for i in final_reviews:
    pos_comment = nltk.pos_tag([i])
    word = pos_comment[0]
    if word[1] == 'NN':
        x.append(word[0])
x

['f',
 'trip',
 'stay',
 'place',
 'september',
 'hotel',
 'front_desk',
 'welcome',
 'house_keep',
 'team',
 'room',
 'clean',
 'responsive',
 'phone',
 'request',
 'towel',
 'need',
 'room',
 'front_desk',
 'listen',
 'issue',
 'name',
 'suba',
 'mgr',
 'duty',
 'provider',
 'mr',
 'thompsongreat_location_excellent',
 'staff',
 'hotel_great_location',
 'excellent',
 'staff',
 'everything',
 'clean',
 'pool',
 'pm',
 'room',
 'clarke_quay_mrt_station',
 'right_next_door',
 'escalator',
 'singapore',
 'train',
 'system',
 'cheap',
 'use',
 'stay',
 'singaporecomfy',
 'hotel',
 'hotel_well_locate',
 'downtown',
 'close_clarke_quay',
 'restaurant',
 'option',
 'walk_distance',
 'subway',
 'across_street',
 'hotel',
 'executive_floor',
 'room',
 'bed_comfy',
 'pillow',
 'select',
 'desk',
 'wifi',
 'bathroom',
 'breakfast',
 'floor',
 'view',
 'city',
 'choice',
 'egg',
 'buffet',
 'cold',
 'cut',
 'salad',
 'yoghurt',
 'breakfast',
 'amenity',
 'room',
 'kettle',
 'nespresso_coffee',
 't

In [20]:
final_reviews = x

In [21]:
dictionary = corpora.Dictionary([final_reviews])

In [22]:
doc_term_matrix = [dictionary.doc2bow([doc]) for doc in final_reviews]

In [23]:
'''
coherence = []
for k in range(3,11):
    print('Round: '+str(k))
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=k, id2word = dictionary, passes=40,\
                   iterations=200, chunksize = 10000, eval_every = None)
    
    cm = gensim.models.coherencemodel.CoherenceModel(model=ldamodel, texts=final_reviews,\
                                                     dictionary=dictionary, coherence='c_v')
    coherence.append((k,cm.get_coherence()))
'''

"\ncoherence = []\nfor k in range(3,11):\n    print('Round: '+str(k))\n    Lda = gensim.models.ldamodel.LdaModel\n    ldamodel = Lda(doc_term_matrix, num_topics=k, id2word = dictionary, passes=40,                   iterations=200, chunksize = 10000, eval_every = None)\n    \n    cm = gensim.models.coherencemodel.CoherenceModel(model=ldamodel, texts=final_reviews,                                                     dictionary=dictionary, coherence='c_v')\n    coherence.append((k,cm.get_coherence()))\n"

In [24]:
'''
x_val = [x[0] for x in coherence]
y_val = [x[1] for x in coherence]
'''

'\nx_val = [x[0] for x in coherence]\ny_val = [x[1] for x in coherence]\n'

In [25]:
'''
plt.plot(x_val,y_val)
plt.scatter(x_val,y_val)
plt.title('Number of Topics vs. Coherence')
plt.xlabel('Number of Topics')
plt.ylabel('Coherence')
plt.xticks(x_val)
plt.show()
'''

"\nplt.plot(x_val,y_val)\nplt.scatter(x_val,y_val)\nplt.title('Number of Topics vs. Coherence')\nplt.xlabel('Number of Topics')\nplt.ylabel('Coherence')\nplt.xticks(x_val)\nplt.show()\n"

In [26]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=40,\
               iterations=200,  chunksize = 10000, eval_every = None, random_state=0)

In [27]:
ldamodel.show_topics(5, num_words=10, formatted=False)

[(0,
  [('breakfast', 0.04719914),
   ('excellent', 0.03820803),
   ('time', 0.03036805),
   ('food', 0.030072052),
   ('use', 0.021321738),
   ('view', 0.019359602),
   ('book', 0.015795587),
   ('visit', 0.015600491),
   ('bathroom', 0.014950731),
   ('recommend', 0.01443949)]),
 (1,
  [('singapore', 0.060804326),
   ('night', 0.033714913),
   ('day', 0.032047506),
   ('need', 0.022357717),
   ('area', 0.017826013),
   ('swissotel', 0.014340849),
   ('lot', 0.01409721),
   ('bed', 0.01348591),
   ('choice', 0.0133793345),
   ('offer', 0.012774463)]),
 (2,
  [('staff', 0.09473092),
   ('service', 0.05721405),
   ('pool', 0.0415259),
   ('clean', 0.031589426),
   ('river', 0.01868116),
   ('walk', 0.016471967),
   ('mrt', 0.01599193),
   ('experience', 0.015320849),
   ('perfect', 0.01434396),
   ('right', 0.013207028)]),
 (3,
  [('room', 0.17678952),
   ('stay', 0.120637335),
   ('check', 0.027530793),
   ('place', 0.026550187),
   ('bar', 0.01976542),
   ('enjoy', 0.01588922),
   ('w

In [28]:
topic_data =  pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary, mds = 'pcoa')
pyLDAvis.display(topic_data)